<a href="https://colab.research.google.com/github/zeton24/gsn_iot_anomalies_detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notatki, linki i importy

In [1]:
! pip install pytorch-lightning
! pip install polars --upgrade
! pip install -U 'wandb>=0.12.10'
! pip install hydra-core --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 14.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

In [1]:
import os
import gdown
import torch
import pytorch_lightning as pl
import polars
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from torchmetrics.functional import accuracy, precision, recall, f1_score
import torch.nn.functional as F
from hydra.utils import instantiate
from yaml import safe_load
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import wandb

Convolutional (Conv2d)
https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d

Average Pooling https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html#torch.nn.AvgPool2d

Normalization
https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#torch.nn.BatchNorm1d

Spatial Dropout
https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html#torch.nn.Dropout2d

Fully connected layer -> nn.Linear()


koszt - cross entropy

aktywacja lineara - relu

aktywacja po 16 neuronach - softmax

zrobili L1, L2 i dropout, ale nie wchodziłam w to dokładnie

batch - 64, 128 dawały najlepsze wyniki

eksperymentalnie sprawdzili, że 100 epok daje zbieżność

w adamie dali 0.0001 learning rate


# Hydra

In [22]:
!git clone https://github.com/zeton24/gsn_iot_anomalies_detection.git github

Cloning into 'github'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 36), reused 15 (delta 3), pack-reused 0
Receiving objects: 100% (75/75), 3.40 MiB | 6.33 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [23]:
!mv github/conf conf
!rm github -r

In [4]:
with open('conf/dataset/mqtt.yaml', 'rt') as f:
    conf_data = safe_load(f.read())
with open('conf/anomalyClassifier.yaml', 'rt') as f:
    conf_model = safe_load(f.read())

# W&B

In [5]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Inicjalizacja wandb logger
wandb_logger = WandbLogger(project='test W&B', job_type='train')

# Lightning

### Callbacki

In [7]:
MODEL_CKPT_PATH = 'checkpoints/'
MODEL_CKPT = 'model-{epoch:02d}-{val_loss:.2f}'

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=MODEL_CKPT_PATH,
    filename=MODEL_CKPT,
    save_top_k=3,
    mode='min')

In [8]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=5,
   verbose=False,
   mode='min'
)

In [9]:
callbacks = [early_stop_callback, checkpoint_callback]

### dane

In [10]:
class LazyDataset(torch.utils.data.Dataset):
  def __init__(self, inputs: torch.tensor, labels: torch.tensor, n_classes: int):
    self.inputs = inputs
    self.labels = labels
    self.n_classes = n_classes

  def __len__(self):
    return self.labels.shape[0]

  def __getitem__(self, idx):
    temp = torch.zeros(self.n_classes)
    temp[self.labels[idx].item()] = 1
    return self.inputs[idx], temp

In [11]:
class IoTDataModule(pl.LightningDataModule):
  def __init__(self, file_id: str, file_name: str, batch_size: int=64, binary_classification: bool=False):
    super().__init__()
    self.file_id = file_id
    self.file_name = file_name
    self.batch_size = batch_size
    self.binary_classification = binary_classification

    if self.binary_classification:
      self.n_classes = 2
      self.mapping = {"Normal": 0, "Anomaly": 1}
    else:
      self.n_classes = 5
      self.mapping = {'Normal': 0, 'Mirai': 1, 'DoS': 2, 'Scan': 3, 'MITM ARP Spoofing': 4}

  def prepare_data(self):
    if not os.path.exists(self.file_name):
      gdown.download(id=self.file_id, output=self.file_name)

  def setup(self,stage = None):
    data = torch.reshape(polars.read_csv(self.file_name, columns = range(64)).cast(polars.Float32).to_torch(), (-1, 1, 64))

    if self.binary_classification:
      labels = torch.reshape(polars.read_csv(self.file_name, columns = [64])['Label'].replace(self.mapping, return_dtype=polars.UInt8).to_torch(), (-1, 1))
    else:
      #labels = torch.reshape(polars.read_csv(self.file_name, columns = [65])['Cat'].replace(self.mapping, return_dtype=polars.UInt8).to_torch(), (-1, 1))
      labels = polars.read_csv(self.file_name, columns = [65])['Cat'].replace(self.mapping, return_dtype=polars.UInt8).to_torch()
    dataset = LazyDataset(data, labels, self.n_classes)
    del data, labels

    # Train, test i val.
    l = len(dataset)
    train_and_val_size = int(l * .75)
    dataset, self.test_dataset = random_split(dataset, [train_and_val_size, l - train_and_val_size])

    ll = len(dataset)
    train_size = int( ll * .9)
    self.train_dataset, self.val_dataset = random_split(dataset, [train_size, ll - train_size])
    del dataset

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size)

### model

In [12]:
class Model1D(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.c1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same')
        self.relu1 = nn.ReLU()
        self.norm1 = nn.BatchNorm1d(32)
        self.pool1 = nn.AvgPool1d(2)
        self.drop1 = nn.Dropout(p=0.05)

        self.convblock2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        # Nazwa bloku do rozważenia, taka mi się wymyśliła, ale nie upieram się przy niej.
        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
            # Bez aktywacji na końcu, bo softmax się doda automatycznie razem z cross entropy.
        )

    def forward(self, x):
      x = self.convblock1(x)
      x = self.convblock2(x)
      x = self.convblock3(x)
      x = self.convblock4(x)
      x = self.evaluator(x)
      return x

In [13]:
class AnomalyClassifier(pl.LightningModule):

  def __init__(self, lr, model_type=1, binary_classification=False):
    super().__init__()
    self.save_hyperparameters()
    self.lr = lr
    self.current_epoch_training_loss = torch.tensor(0.0)
    self.training_step_outputs = []
    self.validation_step_outputs = []

    self.num_classes = 2 if binary_classification else 5

    if model_type == 1:
      self.model = Model1D(self.num_classes)
    elif model_type == 2:
      self.model = Model2D(self.num_classes)
    else:
      self.model = Model3D(self.num_classes)

  def forward(self, x):
    return self.model(x)

  def compute_loss(self, x, y):
    #print(x)
    return F.cross_entropy(x, y)

  def common_step(self, batch, batch_idx):
    x, y = batch
    outputs = self(x)
    loss = self.compute_loss(outputs,y)
    return loss, outputs, y

  def common_test_valid_step(self, batch, batch_idx):
    loss, outputs, y = self.common_step(batch, batch_idx)
    preds = torch.argmax(outputs, dim=1)
    z = torch.argmax(y, dim=1)
    acc = accuracy(preds, z, num_classes = self.num_classes, task="multiclass")
    prec = precision(preds, z, num_classes = self.num_classes, task="multiclass")
    rec = recall(preds, z, num_classes = self.num_classes, task="multiclass")
    f1 = f1_score(preds, z, num_classes = self.num_classes, task="multiclass")
    return loss, acc, prec, rec, f1

  def training_step(self, batch, batch_idx):
    loss, outputs, y = self.common_step(batch, batch_idx)
    self.training_step_outputs.append(loss)
    preds = torch.argmax(outputs, dim=1)
    z = torch.argmax(y, dim=1)
    acc = accuracy(preds, z, num_classes = self.num_classes, task="multiclass")
    #print(f'train_loss: {loss}, train_acc: {acc}')
    self.log_dict({"train_loss": loss, "train_accuracy": acc}, on_step = False, on_epoch = True, prog_bar = True) # logger=True?
    return {'loss':loss}

  def on_train_epoch_end(self):
    outs = torch.stack(self.training_step_outputs)
    self.current_epoch_training_loss = outs.mean()
    #print(f'train loss: {self.current_epoch_training_loss}')
    self.training_step_outputs.clear()

  def validation_step(self, batch, batch_idx):
    loss, acc, _, _, _ = self.common_test_valid_step(batch, batch_idx)
    self.validation_step_outputs.append(loss)

    self.log_dict({'val_loss': loss, 'val_acc': acc}, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    #print(f'val_loss: {loss}, val_acc: {acc}')
    return {'val_loss':loss, 'val_acc': acc}

  def on_validation_epoch_end(self):
    outs = torch.stack(self.validation_step_outputs)
    avg_loss = outs.mean()
    #self.logger.experiment.add_scalars('train and vall losses', {'train': self.current_epoch_training_loss.item() , 'val': avg_loss.item()}, self.current_epoch)

    self.log('train', self.current_epoch_training_loss.item(), on_epoch=True, logger=True)
    self.log('val', avg_loss.item(), on_epoch=True, logger=True)

    self.validation_step_outputs.clear()

  def test_step(self, batch, batch_idx):
    loss, acc, prec, rec, f1 = self.common_test_valid_step(batch, batch_idx)

    self.log_dict({'test_loss': loss, 'test_acc': acc}, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    self.log_dict({'precision': prec, 'recall': rec, 'f1_score':  f1}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    return {'test_loss': loss, 'test_acc': acc, 'precision': prec, 'recall': rec, 'f1_score':  f1}

  def configure_optimizers(self):
    optimizer =  torch.optim.Adam(self.parameters(), lr=self.lr)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    return [optimizer], [lr_scheduler]

### logger

In [ ]:
#from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
#logger = TensorBoardLogger("lightning_logs", name="iot_model")

### trening

In [14]:
dm = instantiate(conf_data)
dm.prepare_data()
dm.setup()

Downloading...
From (original): https://drive.google.com/uc?id=1zCd1BLTtF2JVqz-h-ydUTmpQJIjkBsNg
From (redirected): https://drive.google.com/uc?id=1zCd1BLTtF2JVqz-h-ydUTmpQJIjkBsNg&confirm=t&uuid=ddd76556-4fde-47eb-9cdf-9fdbcfd8e2ed
To: /content/mqtt.csv
100%|██████████| 1.31G/1.31G [00:12<00:00, 103MB/s]


In [15]:
classifier = instantiate(conf_model)
#classifier = AnomalyClassifier.load_from_checkpoint(checkpoint_callback.best_model_path)
trainer = pl.Trainer(accelerator="auto", max_epochs=10, precision=32, logger=wandb_logger, callbacks=callbacks)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(classifier, dm)

wandb: Currently logged in as: zofia_jethon (jethon). Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type    | Params
----------------------------------
0 | model | Model1D | 742 K 
----------------------------------
742 K     Trainable params
0         Non-trainable params
742 K     Total params
2.971     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [17]:
trainer.test(classifier, dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         f1_score          │    0.9394524693489075     │
│         precision         │    0.9394524693489075     │
│          recall           │    0.9394524693489075     │
│      test_acc_epoch       │    0.9394524693489075     │
│      test_loss_epoch      │    0.19484785199165344    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.19484785199165344,
  'test_acc_epoch': 0.9394524693489075,
  'precision': 0.9394524693489075,
  'recall': 0.9394524693489075,
  'f1_score': 0.9394524693489075}]

# W&B c.d.

In [18]:
wandb.finish()

epoch,▁▁▃▃▅▅▆▆█
f1_score,▁
precision,▁
recall,▁
test_acc_epoch,▁
test_acc_step,▄▅▆▆▇▇▆▆▆▄▅▂▅▄▄▁▇▅▄▂▄█▂▆▁▄▆▂▅▆▅▄▅▄▆▆▃▂▄▂
test_loss_epoch,▁
test_loss_step,▅▅▂▂▂▂▃▂▂▄▅▄▄█▃▅▂▄▅▆▅▁▇▂▆▄▂▅▄▄▃▄▃▄▃▅█▆▄▇
train,▁▅█▇
train_accuracy,█▁▅▄
train_loss,▁█▄▅


In [ ]:
run = wandb.init(project='test W&B', job_type='producer')

artifact = wandb.Artifact('model', type='model')
artifact.add_dir(MODEL_CKPT_PATH)

run.log_artifact(artifact)
run.join()

Odwiedź wyświetloną na wyjściiu komórki stronę, przyjrzyj się wynikom zwizualizowanym przez W&B.

Uwaga poboczna: jeśli chciałbyś/aś pobrać najlepszy checkpoint, aby dalej móc z nim pracować wystarczy, skorzystać z `checkpoint_callback.best_model_path`:



In [ ]:
loaded_model = AnomalyClassifier.load_from_checkpoint(checkpoint_callback.best_model_path)

In [ ]:
print(loaded_model.num_classes)

2


# Modele sieci 1D, 2D, 3D

In [ ]:
class Model1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(128),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.AvgPool1d(2),
            nn.Dropout(p=0.05)
        )

        # Nazwa bloku do rozważenia, taka mi się wymyśliła, ale nie upieram się przy niej.
        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
            # Bez aktywacji na końcu, bo softmax się doda automatycznie razem z cross entropy.
        )

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

In [ ]:
class Model2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.AvgPool2d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),

            # Oni w artykule dają normalizację, ale pytorch wywala błąd.
            # Moim zdaniem słusznie, bo o ile rozumiem ten wzór na normalizację (co jest w dokumentacji),
            # to przy wymiarze wejścia [batch_size, 256, 1, 1] wyjście to tensor zer o takim samym wymiarze.
            # Po prostu normalizacja zmienia nam średnią na 0 (odchylenie standardowe też, ale to nieistotne),
            # a jak mamy 1 element, to zmiana średniej na 0, to zmiana elementu na 0.
            # Więc nawet jak jakoś obejdziemy ten błąd, to wyniki będą bez sensu.
            # Moim zdaniem w artykule jest błąd (a przynajmniej na rysunku, może w implementacji tego nie ma).

            nn.BatchNorm2d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,8,8)) # można ten reshape dać gdzie indziej jak się znajdzie lepsze miejsce
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

In [ ]:
class Model3D(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=32, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(32),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(64),
            nn.AvgPool3d(2),
            nn.Dropout(p=0.05)
        )

        self.convblock3 = nn.Sequential(
            nn.Conv3d(in_channels=64, out_channels=128, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(128),
            nn.Dropout(p=0.05)
        )

        self.convblock4 = nn.Sequential(
            nn.Conv3d(in_channels=128, out_channels=256, kernel_size=5, padding='same'),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(256),
            nn.Dropout(p=0.05)
        )

        self.evaluator = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 16)
        )

    def forward(self, x):
        x = torch.reshape(x, (x.shape[0],1,4,4,4))
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.evaluator(x)
        return x

# Poligon

In [ ]:
%%writefile conf/car.yaml
_target_: __main__.Car

engine:
  _target_: __main__.Engine
  power: 10
  volume: 2
brand: 'Mazda'
speed: 2.

In [ ]:
# Load the config file
with open('conf/car.yaml', 'rt') as f:
    conf = safe_load(f.read())

In [ ]:
print(conf)

In [ ]:
class Engine():
  def __init__(self, power: int, volume: int):
    self.power = power
    self.volume = volume

  def __repr__(self):
    return 'engine with power %d and volume %d'%(self.power, self.volume)

In [ ]:
class Car():
  def __init__(self, engine: Engine, brand: str = 'Ford', speed: float = 1):
    self.brand = brand
    self.x = 0
    self.y = 0
    self.speed = speed
    self.engine = engine

  def drive(self, dx: int = 1, dy: int = 1):
    print('%s went from x=%d y=%d in direction dx=%d dy=%d with speed %f'%(self.brand, self.x, self.y, dx, dy, self.speed))
    self.x += dx*self.speed
    self.y += dy*self.speed

In [ ]:
#conf = DictConfig('conf/car.yaml')
ins_car = instantiate(conf)
ins_car.drive(4,4)
print(ins_car.engine)

Tu testuję różne rzeczy, ale tak, żeby się nie mieszały z resztą